# BUSINESS CHALLENGE

## Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

### Inputs for our solution : company name and their primary website.
### Output : Formatted Brochure


In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants
# For this you need to set up your .env file

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we want to scrape, with internal links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content 
        soup = BeautifulSoup(self.body, 'html.parser') 
        self.title = soup.title.string if soup.title else "No title found" 
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose() 
            self.text = soup.body.get_text(separator="\n", strip=True) 
        else:
            self.text = ""
        
        #We want the links as well which are mentioned on the front page
        links = [link.get('href') for link in soup.find_all('a')] 
        self.links = [link for link in links if link] 

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://huggingface.co/")
print (ed.links)

['/', '/models', '/datasets', '/spaces', '/posts', '/docs', '/enterprise', '/pricing', '/login', '/join', '/deepseek-ai/DeepSeek-V3', '/deepseek-ai/DeepSeek-V3-Base', '/PowerInfer/SmallThinker-3B-Preview', '/black-forest-labs/FLUX.1-dev', '/hexgrad/Kokoro-82M', '/models', '/spaces/JeffreyXiang/TRELLIS', '/spaces/osanseviero/gemini-coder', '/spaces/lllyasviel/iclight-v2', '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On', '/spaces/Qwen/QVQ-72B-preview', '/spaces', '/datasets/agibot-world/AgiBotWorld-Alpha', '/datasets/fka/awesome-chatgpt-prompts', '/datasets/PowerInfer/QWQ-LONGCOT-500K', '/datasets/HuggingFaceTB/finemath', '/datasets/O1-OPEN/OpenO1-SFT', '/datasets', '/join', '/pricing#endpoints', '/pricing#spaces', '/pricing', '/enterprise', '/enterprise', '/enterprise', '/enterprise', '/enterprise', '/enterprise', '/enterprise', '/allenai', '/facebook', '/amazon', '/google', '/Intel', '/microsoft', '/grammarly', '/Writer', '/docs/transformers', '/docs/diffusers', '/docs/safetensors', '/docs

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Trying to code this without LLMs by parsing and analyzing the webpage would be hard!

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
get_links_user_prompt(ed)

'Here is the list of links on the website of https://huggingface.co/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\nLinks (some might be relative links):\n/\n/models\n/datasets\n/spaces\n/posts\n/docs\n/enterprise\n/pricing\n/login\n/join\n/deepseek-ai/DeepSeek-V3\n/deepseek-ai/DeepSeek-V3-Base\n/PowerInfer/SmallThinker-3B-Preview\n/black-forest-labs/FLUX.1-dev\n/hexgrad/Kokoro-82M\n/models\n/spaces/JeffreyXiang/TRELLIS\n/spaces/osanseviero/gemini-coder\n/spaces/lllyasviel/iclight-v2\n/spaces/Kwai-Kolors/Kolors-Virtual-Try-On\n/spaces/Qwen/QVQ-72B-preview\n/spaces\n/datasets/agibot-world/AgiBotWorld-Alpha\n/datasets/fka/awesome-chatgpt-prompts\n/datasets/PowerInfer/QWQ-LONGCOT-500K\n/datasets/HuggingFaceTB/finemath\n/datasets/O1-OPEN/OpenO1-SFT\n/datasets\n/join\n/pricing#endpoints\n/pricing#spaces\n/pricing\n/enterprise\n/enterprise\n/enterpr

### Using OpenAI GPT-4o-mini

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://huggingface.co/")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: Making the brochure!

Assemble all the details into another prompt to GPT4-o

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url) # All the associated links on the website in JSON format, calling API here
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents() 
        ## Getting contents from all the useful links, after filtering from LLM
    return result

In [12]:
# get_all_details("https://huggingface.co/")

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [15]:
# get_brochure_user_prompt("HuggingFace", "https://huggingface.co/")

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co/")

# Hugging Face Company Brochure

## Welcome to Hugging Face! 
The AI community building the future. Hugging Face is a collaborative platform where the machine learning (ML) community can join forces to create, share, and develop models, datasets, and applications. With a goal to democratize machine learning, Hugging Face isn’t just a company — it's a movement.

---

## Company Overview
Hugging Face offers a range of powerful tools and services including:
- **Models**: Access to over 400k models, including state-of-the-art transformers and diffusion models.
- **Datasets**: A growing collection of 100k+ datasets for various ML tasks.
- **Spaces**: A space to run applications and showcase ML projects.
- **Enterprise Solutions**: Tailored services for organizations looking to implement cutting-edge AI with enterprise-grade security (For more, see our [Enterprise Hub](https://huggingface.co/about)).

The company is home to a vibrant community of over 50,000 organizations, including notable names like Google, Microsoft, and Amazon, all leveraging the Hugging Face platform to advance their AI capabilities.

---

## Company Culture
At Hugging Face, we are on a mission to democratize good machine learning, one commit at a time. We are a diverse and inclusive team that values collaboration, innovation, and community contributions. Our culture encourages open communication and knowledge-sharing where everyone has a voice.

Join our community on platforms such as [GitHub](https://github.com/huggingface), [Discord](https://discord.gg/huggingface), and [Twitter](https://twitter.com/huggingface) to get involved!

---

## Careers at Hugging Face
Hugging Face is always looking for talented individuals to join our team. Whether you are an engineer, researcher, or a marketing professional, if you believe in the power of open-source and community-driven ML, we want to hear from you. Check our current job openings [here](https://huggingface.co/join-us).

---

## Why Choose Hugging Face?
- **Open Source Commitment**: Hugging Face has a strong commitment to open-source principles, making powerful ML tools accessible to everyone.
- **Fast Deployment**: With optimized inference endpoints and various compute options, you can smoothly deploy your ML models.
- **Learning and Growth**: Join our community to learn, share, and grow in the field of machine learning.
- **Enterprise Readiness**: With advanced security features and dedicated support, our enterprise offerings make Hugging Face the right choice for organizations looking to lead in AI.

---

## Join Us!
Ready to shape the future of AI with Hugging Face? Sign up for free and start creating, collaborating, and exploring our extensive resources. Together, we can unlock the potential of machine learning for everyone!

[**Sign Up**](https://huggingface.co/join)

For more information, visit our website: [Hugging Face](https://huggingface.co)

--- 

Thank you for considering Hugging Face! Let’s build the future of AI together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Brochure

## The Company

**Hugging Face** is at the forefront of the artificial intelligence revolution, serving as a collaborative community where machine learning enthusiasts come together to build models, datasets, and innovative applications. By providing a platform that fosters collaboration, Hugging Face helps shape the future of AI.

- **Website:** [Hugging Face](https://huggingface.co)
- **Motto:** "The AI community building the future."

## Services Offered
- **Models:** Access to over 400,000 machine learning models.
- **Datasets:** An extensive library of over 100,000 datasets for various ML tasks.
- **Spaces:** A section dedicated to creating and sharing ML applications.
- **Enterprise Solutions:** Robust tools for businesses, including security features and dedicated support.

## Customer Base
More than **50,000 organizations** utilize Hugging Face's services, including high-profile companies such as:
- **Google**
- **Microsoft**
- **Amazon Web Services**
- **Meta**

## Open Source Community
Hugging Face embraces an open-source model, enabling developers to contribute to state-of-the-art machine learning technologies like **Transformers**, **Diffusers**, and **Tokenizers**.

## Company Culture
At Hugging Face, the culture is centered around **collaboration**, **innovation**, and a commitment to open-source principles. The community-driven environment encourages knowledge sharing and creativity, allowing team members to grow and contribute in a meaningful way.

## Career Opportunities
Hugging Face offers a variety of career opportunities in fields such as:
- Machine Learning Engineering
- Software Development
- Product Management
- Marketing and Communications

The company seeks passionate individuals eager to be part of a thriving community dedicated to shaping the future of artificial intelligence.

### Join Us!
If you are interested in being part of this innovative team, check out their [career page](https://huggingface.co/careers).

## Pricing
Hugging Face provides various pricing models suitable for individual developers, small teams, and large enterprises:
- **Free Tier:** Access to collaborate on ML models and datasets.
- **Pro Account:** Enhance your capabilities for **$9/month**.
- **Enterprise Hub:** Starting at **$20/user/month** offering advanced features and dedicated support.

## Conclusion
Hugging Face stands at the crossroads of technology and community, continually pushing the boundaries of what is possible in AI. Whether you’re a prospective customer, investor, or recruit, the future with Hugging Face is bright, innovative, and collaborative. 

---

### Connect on Social Media
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://huggingface.co/discord)

For questions or more information, visit our [help center](https://huggingface.co/help). 

**Start your journey with Hugging Face today!**